In [0]:
%pip install pdfplumber

In [0]:
#this is one time loading. Please place into the Volume folder /Volumes/nyctaxi/00_landing/data_sources/tmp/ the file from that link -- https://www.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf
import pdfplumber
import re

rows = []
with pdfplumber.open(
    "/Volumes/nyctaxi/00_landing/data_sources/tmp/trip_record_user_guide.pdf"
) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                rows.append(row)

# Clean column names: remove invalid characters and replace spaces with underscores
def clean_col(col):
    return re.sub(r'[ ,;{}()\n\t=]', '_', col.strip())

header = [clean_col(col) for col in rows[0]]
data = rows[1:]

df = spark.createDataFrame(data, header)
df.write.mode("overwrite").saveAsTable("nyctaxi.01_bronze.trip_record_user_guide")
